In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
globalAA = pd.read_csv('queries_results/clustering/A.csv', header = 0, sep=' *, *')
# globalA["Time"]=pd.to_datetime(globalA['MonthYear'], format='%Y%m').dt.strftime('%Y-%m')

/home/micha/anaconda3/envs/geo_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
# droping events without Actor 1 Country Code
nan = globalAA[globalAA.Actor1CountryCode.isna()].iloc[0].name
globalAA = globalAA.drop(globalAA.index[nan])

In [4]:
import geopandas

world = geopandas.read_file(
    geopandas.datasets.get_path('naturalearth_lowres'))

# add lacking country codeds
world.loc[world.name=='France','iso_a3'] = 'FRA'
world.loc[world.name=='Norway','iso_a3'] = 'NOR'
world.loc[world.name=='N. Cyprus','iso_a3'] = 'CYP'
world.loc[world.name=='Kosovo','iso_a3'] = 'FRY'

In [5]:
# droping events without iso_a3 code
globalAA = world.merge(globalAA, left_on='iso_a3', right_on='Actor1CountryCode')

In [6]:
globalA = globalAA[['Events','sumNumMentions','materialConfCoop','verbalConfCoop','avgAvgTone','avgGoldstein','fightCount','expressCount']]

In [7]:
points = globalA
points = points.to_numpy()
points

array([[1.15700000e+03, 5.75900000e+03, 7.69230769e-02, ...,
        1.34433881e+00, 3.20000000e+01, 3.90000000e+01],
       [2.36800000e+03, 1.08160000e+04, 1.01351351e-01, ...,
        1.40950169e+00, 7.10000000e+01, 3.10000000e+01],
       [9.12140000e+04, 4.06209000e+05, 1.16769356e-01, ...,
        9.38683755e-01, 5.13200000e+03, 2.37700000e+03],
       ...,
       [4.22200000e+03, 2.70500000e+04, 1.07531975e-01, ...,
        1.13256750e+00, 1.84000000e+02, 1.09000000e+02],
       [1.46300000e+03, 6.43500000e+03, 1.57894737e-01, ...,
        2.25632262e-01, 1.18000000e+02, 1.60000000e+01],
       [1.86300000e+03, 8.49000000e+03, 1.43853999e-01, ...,
        1.05394525e+00, 1.49000000e+02, 2.00000000e+01]])

## Find clusters using [kMeans()](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

In [8]:
#klasa KMeans z sklearn, konstruktor przyjmuje liczbę klastrów i sposób inicjalizacji centroidów, random_state=1 za
#każdym razem te same punkty startowe
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, init='random', random_state=1)
km = kmeans.fit(points)

In [9]:
#funkcja predict zwaraca etykiety klastru do jakiego został przypisany obiekt
clusters_id = kmeans.predict(points)
clusters_id

array([8, 3, 9, 7, 5, 3, 8, 4, 1, 0, 3, 5, 1, 0, 3, 3, 3, 6, 3, 0, 0, 8,
       4, 3, 2, 0, 0, 1, 3, 8, 3, 5, 3, 0, 8, 1, 3, 8, 9, 5, 5, 0, 0, 8,
       8, 3, 0, 8, 8, 3, 4, 3, 8, 0, 3, 3, 8, 3, 8, 3, 8, 8, 8, 8, 3, 3,
       3, 8, 5, 8, 9, 1, 8, 2, 3, 0, 0, 1, 1, 5, 5, 9, 0, 8, 3, 0, 8, 0,
       0, 0, 2, 8, 4, 1, 8, 0, 4, 2, 8, 3, 8, 6, 2, 0, 1, 0, 9, 2, 0, 5,
       3, 5, 5, 3, 9, 0, 2, 9, 5, 0, 2, 3, 1, 1, 0, 4, 2, 8, 1, 9, 0, 6,
       0, 4, 0, 6, 8, 0, 8, 2, 1, 8, 0, 3, 0, 8, 4, 8, 0, 2, 0, 0, 0, 2,
       4, 0, 8, 5, 3, 5, 5, 8, 3], dtype=int32)

In [10]:
cluster_map = pd.DataFrame()
cluster_map['data_index'] = globalAA['Actor1CountryCode']
cluster_map['cluster'] = km.labels_

### Data standardisation with [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

In [11]:
##jeśli dane są w jakiś sposób skorelowane to znaczy, że występuje redundancja danych, te dane nic nowego nie wnoszą
##są niepotrzebne

## if necessary to scale data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_points = scaler.fit_transform(points)
scaled_points

array([[-0.26697661, -0.29005287, -0.92985533, ...,  0.58796341,
        -0.24702803, -0.22412185],
       [-0.24927919, -0.27285517, -0.45621869, ...,  0.67467861,
        -0.23979039, -0.22907104],
       [ 1.04910616,  1.07178561, -0.15728101, ...,  0.0481398 ,
         0.69943209,  1.22228026],
       ...,
       [-0.22218505, -0.21764707, -0.33638337, ...,  0.30614971,
        -0.2188198 , -0.18081639],
       [-0.26250476, -0.28775395,  0.64009366, ..., -0.90075022,
        -0.23106811, -0.23835078],
       [-0.2566592 , -0.28076537,  0.36785962, ...,  0.20152351,
        -0.22531512, -0.23587618]])

## Find clusters using [kMeans()](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) on standardised data

In [12]:
#klasa KMeans z sklearn, konstruktor przyjmuje liczbę klastrów i sposób inicjalizacji centroidów, random_state=1 za
#każdym razem te same punkty startowe
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, init='random', random_state=1)
km = kmeans.fit(scaled_points)

In [13]:
#funkcja predict zwaraca etykiety klastru do jakiego został przypisany obiekt
clusters_id = kmeans.predict(scaled_points)
clusters_id

array([5, 5, 7, 0, 5, 1, 4, 3, 3, 2, 4, 6, 6, 3, 9, 3, 1, 7, 4, 3, 4, 6,
       4, 1, 3, 2, 4, 3, 3, 1, 3, 9, 4, 4, 4, 3, 4, 3, 7, 5, 5, 3, 3, 3,
       9, 5, 4, 9, 1, 6, 2, 6, 5, 5, 5, 2, 3, 3, 5, 6, 9, 9, 3, 5, 3, 3,
       3, 4, 3, 2, 7, 2, 1, 2, 2, 3, 5, 2, 1, 1, 4, 8, 1, 1, 5, 9, 1, 3,
       5, 4, 5, 4, 2, 4, 1, 4, 2, 6, 4, 5, 1, 8, 6, 3, 3, 5, 7, 2, 3, 3,
       3, 3, 5, 1, 7, 3, 3, 7, 3, 5, 5, 1, 5, 4, 5, 3, 3, 4, 5, 7, 5, 7,
       3, 5, 5, 7, 1, 5, 9, 4, 3, 1, 3, 4, 3, 5, 3, 9, 6, 4, 3, 3, 5, 5,
       3, 5, 5, 3, 4, 1, 5, 4, 4], dtype=int32)

In [14]:
cluster_map_std = pd.DataFrame()
cluster_map_std['data_index'] = globalAA['Actor1CountryCode']
cluster_map_std['cluster'] = km.labels_